# Pytest

---

## 설치 및 시작하기


### 간단한 첫번째 테스트

In [ ]:
def func(x):
    return x + 1

def test_answer():
    assert func(3) == 5

```cmd
pytest
```

현재 디렉토리 및 하위 디렉토리의 test_*.py 파일을 모두 테스트

### 특정 예외 발생 확인

In [ ]:
import pytest

def f():
    raise SystemExit(1)

def test_mytest():
    with pytest.raises(SystemExit):
        f()

### 클래스로 테스트 그룹화

그룹화된 테스트 클래스는 Test* 형태의 이름이어야 함, test* 이름의 메서드를 모두 테스트

그룹화, 클래스 내에만 제공되는 픽스쳐 등에 활용

In [ ]:
class TestClass:
    def test_one(self):
        x = 'this'
        assert 'h' in x
    
    def test_two(self):
        x = "hello"
        assert hasattr(x, "check")

---

## 프로젝트 레이아웃



대략적으로 다음과 같이 설계하는 게 좋음, [이유](https://blog.ionelmc.ro/2014/05/25/python-packaging/#the-structure)

```
setup.py
src/
    mypkg/
        __init__.py
        app.py
        view.py
tests/
    __init__.py
    foo/
        __init__.py
        test_view.py
    bar/
        __init__.py
        test_view.py

```

setup.py를 실행해 mypkg를 파이썬 환경에 설치함 -> tests 내의 테스트 코드 수행

---

## 사용법 및 호출

### 종료코드

pytest 실행 시
- 0: 테스트 성공적
- 1: 일부 테스트 실패
- 2: 사용자 테스트 중단
- 3: 테스트 진행 중 내부 오류
- 4: pytest 잘못된 인자
- 5: 테스트 없음

### 픽스쳐 목록

pytest --fixtures

### N회 실패 시 중지

실패 발견 시 종료 : pytest -x
    
실패 N회 발견 시 종료 : pytest --maxfail=2

### 테스트 범위 선택

모듈 : pytest test_module.py

패키지 : pytest testing/

---

## Fixture

### 테스트의 단계

- Arrange: 준비
- Act: 수행
- Assert: 확인
- Cleanup: 격리

### Fixture 기본

간단하게 말하면 의존성 주입

In [ ]:
import pytest


class Fruit:
    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return self.name == other.name


# [ Arrange ]
@pytest.fixture
def my_fruit():
    return Fruit("apple")

@pytest.fixture
def fruit_basket(my_fruit):  # fixture 중첩 가능ㅠ
    return [Fruit("banana"), my_fruit]


# [ Act ]
def test_my_fruit_in_basket(my_fruit, fruit_basket):  # fixture 여러개 사용 가능
    # [ Assert ]
    assert my_fruit in fruit_basket

def test_my_fruit_in_basket_other(my_fruit, fruit_basket):  # fixture 재사용 가능
    # [ Assert ]
    assert my_fruit in fruit_basket
# [ Cleanup ]

### Fixture 범위

Fixture 반환 인스턴스의 생명 주기 선택 가능

종류
- function : 테스트 종료 후 파괴
- class : 클래스 마지막 테스트 종료 후 파괴
- module : 모듈 마지막 테스트 종료 후 파괴
- package : 패키지 마지막 테스트 종료 후 파괴
- session : 전체 테스트 종료 후 파괴

In [ ]:
# [ Function 수준 ] Pass

In [ ]:
# [ Class 수준 ]

@pytest.fixture
def order():
    return []

@pytest.fixture
def outer(order, inner):
    order.append("outer")

@pytest.fixture(scope="class")  # 여기
def inner(order):
    order.append("one")

class TestOne:
    @pytest.fixture  # 또는 클래스 내에 정의하여
    def inner(self, order):
        order.append("one")

    def test_order(self, order, outer):
        assert order == ["one", "outer"]


In [ ]:
# [ Module 수준 ]

@pytest.fixture(scope="module")
def smtp_connection():
    return smtplib.SMTP("smtp.gmail.com", 587, timeout=5)

def test_ehlo(smtp_connection):
    response, msg = smtp_connection.ehlo()
    assert response == 250
    assert b"smtp.gmail.com" in msg
    assert 0

def test_noop(smtp_connection):
    response, msg = smtp_connection.noop()
    assert response == 250
    assert 0

In [ ]:
# [ Package 수준 ]

# 패키지 내에 conftest.py를 정의하면
# 해당 내부에 정의된 모든 fixture가 패키지 내부에서 활용 가능

In [ ]:
# [ Session 수준 ]

@pytest.fixture(scope="session")
def order():
    return []

### Autouse

인자로 넣어야 픽스쳐가 호출되는데, 매번 넣기 귀찮고 그래도 호출은 되어야 하는데 어쩌냐? => autouse

In [ ]:
@pytest.fixture
def first_entry():
    return "a"

@pytest.fixture
def order(first_entry):
    return []

@pytest.fixture(autouse=True)  # 여기
def append_first(order, first_entry):
    return order.append(first_entry)

# 아래의 두 테스트에서 append_first를 호출하지 않았더라도 자동으로 호출됨
def test_string_only(order, first_entry):
    assert order == [first_entry]

def test_string_and_int(order, first_entry):
    order.append(2)
    assert order == [first_entry, 2]

In [ ]:
class TestLandingPageSuccess:
    # 이렇게 씀으로써 하위 타 테스트 이전에 assert를 수행
    @pytest.fixture(scope="class", autouse=True)
    def login(self, driver, base_url, user):
        driver.get(urljoin(base_url, "/login"))
        page = LoginPage(driver)
        page.login(user)

    def test_name_in_header(self, landing_page, user):
        assert landing_page.header == f"Welcome, {user.name}!"

    def test_sign_out_button(self, landing_page):
        assert landing_page.sign_out_button.is_displayed()

    def test_profile_link(self, landing_page, user):
        profile_href = urljoin(base_url, f"/profile?id={user.profile_id}")
        assert landing_page.profile_link.get_attribute("href") == profile_href

### Fixture의 실행 순서

session -> package -> module -> class -> function 순서로 fixture가 실행됨

### 테스트 컨텍스트

테스트 중에 생겨난 뒤처리 어쩔겨? yield로 하세요

In [ ]:
import pytest

from emaillib import Email, MailAdminClient


@pytest.fixture
def mail_admin():
    return MailAdminClient()


@pytest.fixture
def sending_user(mail_admin):
    user = mail_admin.create_user()
    yield user
    mail_admin.delete_user(user)


@pytest.fixture
def receiving_user(mail_admin):
    user = mail_admin.create_user()
    yield user
    mail_admin.delete_user(user)


def test_email_received(receiving_user, sending_user):
    email = Email(subject="Hey!", body="How's it going?")
    sending_user.send_email(email, receiving_user)
    assert email in receiving_user.inbox


### 픽스쳐 정의의 올바른 방식

기능별로 분리해서 사용하세요

In [ ]:
from uuid import uuid4
from urllib.parse import urljoin

from selenium.webdriver import Chrome
import pytest

from src.utils.pages import LoginPage, LandingPage
from src.utils import AdminApiClient
from src.utils.data_types import User


@pytest.fixture
def admin_client(base_url, admin_credentials):
    return AdminApiClient(base_url, **admin_credentials)


@pytest.fixture
def user(admin_client):
    _user = User(name="Susan", username=f"testuser-{uuid4()}", password="P4$$word")
    admin_client.create_user(_user)
    yield _user
    admin_client.delete_user(_user)


@pytest.fixture
def driver():
    _driver = Chrome()
    yield _driver
    _driver.quit()


@pytest.fixture
def login(driver, base_url, user):
    driver.get(urljoin(base_url, "/login"))
    page = LoginPage(driver)
    page.login(user)


@pytest.fixture
def landing_page(driver, login):
    return LandingPage(driver)


def test_name_on_landing_page_after_login(landing_page, user):
    assert landing_page.header == f"Welcome, {user.name}!"

### 픽스쳐 여러개 만들기

In [ ]:
@pytest.fixture(params=[1, 2, 3])
def make_double_value(request):
    return (request.param, request.param * 2)


def test_double_value(make_double_value):
    assert make_double_value[1] == (make_double_value[0] * 2 + 1)

### 여기부터 이해가 어렵다

https://docs.pytest.org/en/6.2.x/fixture.html#fixtures-can-introspect-the-requesting-test-context